## Softmax derivative 

In [ ]:
# https://towardsdatascience.com/word2vec-with-pytorch-implementing-original-paper-2cd7040120b0

In [1]:
import numpy as np

In [100]:
# ax + b = scores
# Lets assume that the second class is correct, but our model predicts the third 
correct_class_idx = 1
scores = np.array([1., 2., 3.])


In [101]:
def sigmoid(x):
    return 1. / (1. + np.exp(-x))

def softmax(scores):
    denom = np.sum(np.exp(scores))
    return np.exp(scores) / denom
    

In [102]:
probs = softmax(scores)
probs

array([0.09003057, 0.24472847, 0.66524096])

In [103]:
Loss = probs[correct_class_idx]
Loss # OK lets find derivative with respect to scores

0.24472847105479767

In [104]:
derivative = probs - 1
derivative

array([-0.90996943, -0.75527153, -0.33475904])

In [105]:
upd_scores = scores - derivative
upd_scores # Мне кажется что это неправильно, типо должно было только у одного класса увеличиться а у остальных уменьшиться 

array([1.90996943, 2.75527153, 3.33475904])

In [30]:
# Наверное правильнее будет так, что ты отнимаешь единицу только у правильного класса 
# типо 
derivative_correct = probs.copy()
derivative_correct[correct_class_idx] -= 1
derivative_correct

array([ 0.09003057, -0.75527153,  0.66524096])

In [132]:
correct_upd = scores - derivative_correct
print(scores, scores[np.newaxis, :].shape)
print(correct_upd)

[0.81993885 3.51054306 1.66951809] (1, 3)
[0.72990828 4.26581459 1.00427713]


In [136]:
import torch.nn as nn
import torch
import torch.optim as optim
scores = np.array([1., 2., 3.])
optimizer = optim.SGD([input], lr=1.0)
loss = nn.CrossEntropyLoss()
input = torch.from_numpy(scores[np.newaxis, :])
input.requires_grad = True


print("INPUT:", input)
target = torch.tensor([1], dtype=torch.long)
output = loss(input, target)
output.backward()
print("GRAD:", input.grad)
print("PARAMETERs before : ", optimizer.param_groups)
optimizer.step()
print("INPUT UPD?", input, input - input.grad)
print("PARAMETERs after : ", optimizer.param_groups)


INPUT: tensor([[1., 2., 3.]], dtype=torch.float64, requires_grad=True)
GRAD: tensor([[ 0.0900, -0.7553,  0.6652]], dtype=torch.float64)
PARAMETERs before :  [{'params': [tensor([[1., 2., 3.]], dtype=torch.float64, requires_grad=True)], 'lr': 1.0, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False}]
INPUT UPD? tensor([[1., 2., 3.]], dtype=torch.float64, requires_grad=True) tensor([[0.9100, 2.7553, 2.3348]], dtype=torch.float64, grad_fn=<SubBackward0>)
PARAMETERs after :  [{'params': [tensor([[0.9100, 2.7553, 2.3348]], dtype=torch.float64, requires_grad=True)], 'lr': 1.0, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False}]


In [ ]:
# Хорошо, мы поняли что примерно все так и работает. Что суть в том, что надо вычитать единицу только из правильного класса когда берешь производную